# `OTP` traveltime testing

notebook written by Jarvis Yuan, portions adopted from Brian Hill's `Bogota_OTP_Setup.ipynb`

- running **Open Trip Planner** using a customized jython script to parse and write travel times matrices between user specified origin-destination pairs in a `.csv` files.
- doing preliminary analysis on the results
- designed for testing the travel time savings of the TransMiCable for the UC Berkeley URAP project: using...


### quick access
- [file structure](#0)
- [specifying origins and destinations](#1)
- [editing scripts](#2)
- [build the graph and run the travel time scirpts](#3)
- [runing two scripts directly](#4)
- [compare results and parse data](#5)

In [2]:
import pandas as pd
import numpy as np
import warnings
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
from DisplayablePath import DisplayablePath 
# a custom script that allows for display file structures at a given directory

warnings.filterwarnings('ignore')

In [2]:
# locate your data folder, where you want to store generated outputs
output_path = os.path.abspath('/Users/admin/Desktop/bogota_urap/data/OTP_traveltime/')
# locate the OTP folder, where you store the graph building files
otp_path = os.path.abspath('/Users/admin/Desktop/bogota_urap/OTP')
# locate the origin-destination csv folder
od_path = os.path.abspath('/Users/admin/Desktop/bogota_urap/OTP/origin_destinations')

print(os.getcwd())

/Users/admin/Desktop/bogota_urap/notebooks


<a id="0"></a>

## file structure
- refer to this section as you go, make sure your file structure follows the hierarchy below:
    1. ignore `packages/` folder, `.DS_Store`, and `.jython_cache/`, these are system files.
    2. place **osm street network**, **GTFS feeds** in `.zip` format, and downloaded **OTP graph** that will be built in section below, as well as custom **`router-config.json`** under `no_transMiCable/` and `transMiCable/`
    3. place **destination and origin points** in two seperate `.csv` or `.txt` files under `origin_destinations/, which will also be specified in section below.
    4. place **`otp-1.4.0-shaded.jar`**, the OpenTripPlanner docker image under the main directory.
    5. create a `scripts` folder, which will be storing the travel time scripts in section below.
    6. if you would like to change the naming scheme of the folders, make sure you do so consistently throughout this notebook in sections below.
- once you are done, copy the cell below and run it with your `otp_path` in a seperate cell to check for the correct structure.

In [ ]:
paths = DisplayablePath.make_tree(otp_path)
for path in paths:
    print(path.displayable())

---

<a id="1"></a>

## specifying origins and destinations

- below it's creating/editing origin and destination points in `.csv` format, run only once and skip this section for repetetive run of the notebook

In [ ]:
os.chdir(od_path)
print(os.getcwd())

#### origins

In [ ]:
%%bash
cat > points_origins.csv
GEOID,Y,X,Description
1,4.549647,-74.159542,Mirador-Paraiso station
2,4.550401,-74.150533,Manitas station
3,4.555669,-74.147564,Juan Pablo II station
4,4.569755,-74.139307,Portal Tunal

#### destinations

In [ ]:
%%bash
cat > points_destinations.csv
GEOID,Y,X,Description
1,4.549647,-74.159542,Mirador-Paraiso station
2,4.550401,-74.150533,Manitas station
3,4.555669,-74.147564,Juan Pablo II station
4,4.569755,-74.139307,Portal Tunal
5,4.597581,-74.072726,Historic District
6,4.659224,-74.061834,Financial District
7,4.755922,-74.045701,Portal Norte
8,4.575976211,-74.12012724,Calle 40 sur station
9,4.571447,-74.128239,Hospital El Tunal
10,4.696172,-74.138572,Airport
11,4.693246,-74.056159,Calle 106 station
12,4.569166,-74.134377,Parque El Tunal
13,4.632203,-74.157224,Abastos
14,4.615257,-74.069262,Museo Nacional

---

<a id="2"></a>

## editing scripts
- creating/editing the python script for running the travl time computation
- Note that there are some customizable options when making the request object, they are outlined here:
http://dev.opentripplanner.org/javadoc/1.4.0/org/opentripplanner/api/common/RoutingResource.html
- run only once and comment out for repetetive run of the notebook

In [ ]:
os.chdir(os.path.join(otp_path, 'scripts/'))
print(os.getcwd())

In [ ]:
%%bash 
cat > travel_time_no_gondola.py

from org.opentripplanner.scripting.api import OtpsEntryPoint
import os

############ specify parameters here ############################################################
graph_path = os.path.abspath('/Users/admin/Desktop/bogota_urap/OTP')
router_folder = 'no_transMiCable'
output_path = os.path.abspath('/Users/admin/Desktop/bogota_urap/data/OTP_traveltime/')
output_name = 'travel_time_no_transMiCable.csv'

origin_destinatons = os.path.abspath('/Users/admin/Desktop/bogota_urap/OTP/origin_destinations')
origin = os.path.join(origin_destinatons, 'points_origins.csv')
destination = os.path.join(origin_destinatons, 'points_destinations.csv')


maxTime = 7200 
modes = 'WALK,BUS,RAIL,TRANSIT'
clampInitialWait = 0
#################################################################################################

# Instantiate an OtpsEntryPoint 
# --router = the subdirectory under 'otp_path' where your gtfs and osm files are located
otp = OtpsEntryPoint.fromArgs(['--graphs', graph_path, 
                               '--router', router_folder])

# Start timing the code
import time
start_time = time.time()

# Get the default router
router = otp.getRouter(router_folder)


# Create a default request for a given departure time
req = otp.createRequest()
req.setDateTime(2019, 9, 15, 10, 00, 00)  # set departure time (April 4, 2019)
req.setMaxTimeSec(maxTime)                # set a limit to maximum travel time (seconds), 7200s = 2hrs
req.setModes(modes)                       # define transport mode
req.setClampInitialWait(clampInitialWait) # clamp the initial wait time to zero
req.maxWalkDistance = 2000                # set the maximum distance (in meters) the user is willing to walk
#req.walkSpeed = walkSpeed                 # set average walking speed ( meters ?)
# ?ERROR req.setSearchRadiusM(500)      # set max snapping distance to connect trip origin to street network


# Read Points of Destination - The file points.csv contains the columns GEOID, X and Y.
points = otp.loadCSVPopulation(origin, 'Y', 'X')
dests = otp.loadCSVPopulation(destination, 'Y', 'X')


# Create a CSV output
matrixCsv = otp.createCSVOutput()
matrixCsv.setHeader([ 'origin', 'destination', 'walk_distance', 'travel_time', 'boardings' ])

# Start Loop
for origin in points:
  print "Processing origin: ", origin
  req.setOrigin(origin)
  spt = router.plan(req)
  if spt is None: continue

  # Evaluate the SPT for all points
  result = spt.eval(dests)

  # Add a new row of result in the CSV output
  for r in result:
    matrixCsv.addRow([ origin.getStringData('GEOID'), 
    r.getIndividual().getStringData('GEOID'), r.getWalkDistance() , r.getTime(),  r.getBoardings() ])

# Save the result
matrixCsv.save(output_name)
os.rename(os.path.join(os.getcwd(), output_name), 
          os.path.join(output_path, output_name))

print('File successfully saved at ', output_path)

# Stop timing the code
print("Elapsed time was %g seconds" % (time.time() - start_time))

In [ ]:
%%bash 
sed 's/no_transMiCable/transMiCable/' travel_time_no_gondola.py > travel_time_gondola.py

---

<a id="3"></a>

## build the graph and run the travel time scirpts

> first run the script with just the Bogota city's GTFS feeds as a benchmark

In [ ]:
# build graphs under the OTP directory

os.chdir(otp_path)
print(os.getcwd())

In [ ]:
%%bash

java -Xmx10G -jar otp-1.4.0-shaded.jar --build /Users/admin/Desktop/bogota_urap/OTP/no_transMiCable
say graph building done

>> if no change has been made to the script or the router folder (thus the graph), just run the cell below

In [ ]:
%%bash 

/Users/admin/jython2.7.2/bin/jython -J-XX:-UseGCOverheadLimit -J-Xmx10G -Dpython.path=otp-1.4.0-shaded.jar /Users/admin/Desktop/bogota_urap/OTP/scripts/travel_time_no_gondola.py

---

> then run the other script which includes the additional GTFS feed for the transMiCable

In [ ]:
%%bash

java -Xmx10G -jar otp-1.4.0-shaded.jar --build /Users/admin/Desktop/bogota_urap/OTP/transMiCable;

>> if no change has been made to the script or the router folder (thus the graph), just run the cell below

In [ ]:
%%bash 

/Users/admin/jython2.7.2/bin/jython -J-XX:-UseGCOverheadLimit -J-Xmx10G -Dpython.path=otp-1.4.0-shaded.jar /Users/admin/Desktop/bogota_urap/OTP/scripts/travel_time_gondola.py

<a id="4"></a>

## runing two scripts directly
- if no changes are needed to make to the **scripts** and no changes were made to the **router folders** (feeds, osm street maps, etc) run the cell below directly for computing the travel times.
- if changes are needed, refer to the section above

In [3]:
# run scripts under the OTP directory

os.chdir(otp_path)
print(os.getcwd())

/Users/admin/Desktop/bogota_urap/OTP


In [4]:
%%bash 

/Users/admin/jython2.7.2/bin/jython -J-XX:-UseGCOverheadLimit -J-Xmx10G -Dpython.path=otp-1.4.0-shaded.jar /Users/admin/Desktop/bogota_urap/OTP/scripts/travel_time_no_gondola.py
/Users/admin/jython2.7.2/bin/jython -J-XX:-UseGCOverheadLimit -J-Xmx10G -Dpython.path=otp-1.4.0-shaded.jar /Users/admin/Desktop/bogota_urap/OTP/scripts/travel_time_gondola.py

00:01:35.851 INFO (OTPServer.java:39) Wiring up and configuring server.
00:01:35.876 INFO (GraphScanner.java:64) Attempting to automatically register routerIds [no_transMiCable]
00:01:35.876 INFO (GraphScanner.java:65) Graph files will be sought in paths relative to /Users/admin/Desktop/bogota_urap/OTP
00:01:35.878 INFO (GraphService.java:176) Registering new router 'no_transMiCable'
00:01:35.879 INFO (InputStreamGraphSource.java:181) Loading graph...
00:01:36.136 INFO (Graph.java:746) Graph version: MavenVersion(1, 4, 0, , b272f14007c97d769216e9ebab65baad7410cdf5)
00:01:36.136 INFO (Graph.java:747) OTP version:   MavenVersion(1, 4, 0, , b272f14007c97d769216e9ebab65baad7410cdf5)
00:01:36.137 INFO (Graph.java:764) This graph was built with the currently running version and commit of OTP.
00:01:41.129 INFO (Graph.java:731) Main graph read. |V|=243706 |E|=1261205
00:01:43.129 INFO (GraphIndex.java:127) Indexing graph...
00:01:44.282 INFO (GraphIndex.java:595) Clustering stops by geographi

hsqldb.db.HSQLDB4AD417742A.ENGINE INFO dataFileCache open start
hsqldb.db.HSQLDB4AD417742A.ENGINE INFO dataFileCache open start


***

<a id="5"></a>

## compare results and parse data
- merge the before and after output of the travel time matrix and compute difference

In [5]:
# load the travel time matrix output for without the gondola feed
no_gondola = pd.read_csv(os.path.join(output_path, 'travel_time_no_transMiCable.csv'))
no_gondola['travel_time_min'] = no_gondola['travel_time'] / 60

# then load the travel time matrix output with the gondola feed 
w_gondola = pd.read_csv(os.path.join(output_path, 'travel_time_transMiCable.csv'))
w_gondola['travel_time_min'] = w_gondola['travel_time'] / 60

print('Travel time matrix without using TransMiCable')
display(no_gondola.head())
print('Travel time matrix using TransMiCable')
display(w_gondola.head())

Travel time matrix without using TransMiCable


,origin,destination,walk_distance,travel_time,boardings,travel_time_min
0,1,1,36.000000,26,0,0.433333
1,1,2,618.113436,874,1,14.566667
2,1,3,710.165714,1043,1,17.383333
3,1,4,190.639421,962,1,16.033333
4,1,5,855.232278,3665,2,61.083333


Travel time matrix using TransMiCable


,origin,destination,walk_distance,travel_time,boardings,travel_time_min
0,1,1,36.000000,26,0,0.433333
1,1,2,220.488083,414,1,6.900000
2,1,3,123.844256,588,1,9.800000
3,1,4,176.301000,996,1,16.600000
4,1,5,855.233278,3665,2,61.083333


In [6]:
merged = pd.merge(no_gondola, w_gondola, on = ['origin', 'destination'], suffixes=('_no_gondola','_gondola'))
merged['difference_min'] = merged['travel_time_min_no_gondola'] - merged['travel_time_min_gondola']
merged.set_index(['origin', 'destination']).head(14)

walk_distance_no_gondola  travel_time_no_gondola  \
origin destination                                                     
1      1                           36.000000                      26   
       2                          618.113436                     874   
       3                          710.165714                    1043   
       4                          190.639421                     962   
       5                          855.232278                    3665   
       6                         1012.036722                    3769   
       7                          633.426023                    4966   
       8                          284.131090                    1880   
       9                          241.287165                    2004   
       10                         656.420639                    5434   
       11                         334.570549                    3876   
       12                         966.155774                    1615   
       13                         598.373188                    3482   
       14                         309.956639                    3805   

                    boardings_no_gondola  travel_time_min_no_gondola  \
origin destination                                                     
1      1                               0                    0.433333   
       2                               1                   14.566667   
       3                               1                   17.383333   
       4                               1                   16.033333   
       5                               2                   61.083333   
       6                               2                   62.816667   
       7                               2                   82.766667   
       8                               2                   31.333333   
       9                               2                   33.400000   
       10                              2                   90.566667   
       11                              2                   64.600000   
       12                              1                   26.916667   
       13                              1                   58.033333   
       14                              2                   63.416667   

                    walk_distance_gondola  travel_time_gondola  \
origin destination                                               
1      1                        36.000000                   26   
       2                       220.488083                  414   
       3                       123.844256                  588   
       4                       176.301000                  996   
       5                       855.233278                 3665   
       6                      1012.039722                 3770   
       7                       749.673601                 4842   
       8                       400.378669                 1756   
       9                       241.288165                 2004   
       10                      656.420639                 5434   
       11                      450.818128                 3752   
       12                      966.157774                 1616   
       13                      598.374188                 3482   
       14                      309.957639                 3805   

                    boardings_gondola  travel_time_min_gondola  difference_min  
origin destination                                                              
1      1                            0                 0.433333        0.000000  
       2                            1                 6.900000        7.666667  
       3                            1                 9.800000        7.583333  
       4                            1                16.600000       -0.566667  
       5                            2                61.083333        0.000000  
       6                            2                62.833333       -0.016667  
       7 

In [7]:
# writing to file
os.chdir(output_path)
print(os.getcwd())
merged.to_csv('boarding300_noclamp_walkdefault.csv')
os.chdir(otp_path)

/Users/admin/Desktop/bogota_urap/data/OTP_traveltime


In [ ]:
sns.set();
plt.figure(figsize=(15,5))

sns.distplot(no_gondola['travel_time_min'], bins=10, rug=True, hist=True, label='Before TransMiCable')
sns.distplot(w_gondola['travel_time_min'], bins=10, rug=True, hist=True, label='With TransMiCable')
plt.legend()
plt.title('travel time difference before and after')

---

## output comparison
upon closer inspection, the `router-config.json` file isn't functioning as expected during the travel time calculations, through the parameters are displayed in the log, so deciede to only use the setter methods in the scripts for both with gondola and without.
http://dev.opentripplanner.org/javadoc/0.19.0/org/opentripplanner/scripting/api/OtpsRoutingRequest.html
1. `boardingdefault_clamp_walkdefault`: `boardingtime=0`, `walkspeed=default`, `clampInitialWait=0`
2. `boardingdefault_noclamp_walkdefault`: `boardingtime=0`, `walkspeed=default`, `clampInitialWait=-1`
3. `boarding60_clamp_walkdefault`: `boardingtime=60`, `walkspeed=default`, `clampInitialWait=0`
4. `boarding60_noclamp_walkdefault`: `boardingtime=60`, `walkspeed=default`, `clampInitialWait=-1`
5. `boarding300_clamp_walkdefault`: `boardingtime=300`, `walkspeed=default`, `clampInitialWait=0`
6. `boarding300_noclamp_walkdefault`: `boardingtime=300`, `walkspeed=default`, `clampInitialWait=-1`

In [5]:
path_data = os.path.abspath('/Users/admin/Desktop/bogota_urap/data/OTP_traveltime/')

In [6]:
boardingdefault_clamp_walkdefault = pd.read_csv(path_data + '/boardingdefault_clamp_walkdefault/boardingdefault_clamp_walkdefault.csv')
boardingdefault_noclamp_walkdefault = pd.read_csv(path_data + '/boardingdefault_noclamp_walkdefault/boardingdefault_noclamp_walkdefault.csv')
boarding60_clamp_walkdefault = pd.read_csv(path_data + '/boarding60_clamp_walkdefault/boarding60_clamp_walkdefault.csv')
boarding60_noclamp_walkdefault = pd.read_csv(path_data + '/boarding60_noclamp_walkdefault/boarding60_noclamp_walkdefault.csv')
boarding300_clamp_walkdefault = pd.read_csv(path_data + '/boarding300_clamp_walkdefault/boarding300_clamp_walkdefault.csv')
boarding300_noclamp_walkdefault = pd.read_csv(path_data + '/boarding300_noclamp_walkdefault/boarding300_noclamp_walkdefault.csv')

In [7]:
boardingdefault_clamp_walkdefault.head()


,Unnamed: 0,origin,destination,walk_distance_no_gondola,travel_time_no_gondola,boardings_no_gondola,travel_time_min_no_gondola,walk_distance_gondola,travel_time_gondola,boardings_gondola,travel_time_min_gondola,difference_min
0,0,1,1,36.000000,26,0,0.433333,36.000000,26,0,0.433333,0.000000
1,1,1,2,484.622917,1234,1,20.566667,220.488083,423,1,7.050000,13.516667
2,2,1,3,710.165714,1528,1,25.466667,123.844256,597,1,9.950000,15.516667
3,3,1,4,190.639421,1447,1,24.116667,176.301000,1005,1,16.750000,7.366667
4,4,1,5,855.232278,3556,2,59.266667,855.233278,3556,2,59.266667,0.000000


In [8]:
comparison = boardingdefault_clamp_walkdefault[['origin', 'destination', 'difference_min']]
comparison['boardingdefault_noclamp_walkdefault'] = boardingdefault_noclamp_walkdefault['difference_min']
comparison['boarding60_clamp_walkdefault'] = boarding60_clamp_walkdefault['difference_min']
comparison['boarding60_noclamp_walkdefault'] = boarding60_noclamp_walkdefault['difference_min']
comparison['boarding300_clamp_walkdefault'] = boarding300_clamp_walkdefault['difference_min']
comparison['boarding300_noclamp_walkdefault'] = boarding300_noclamp_walkdefault['difference_min']
comparison = comparison.set_index(['origin', 'destination']).rename(columns={'difference_min': 'boardingdefault_clamp_walkdefault'})


In [9]:
comparison

boardingdefault_clamp_walkdefault  \
origin destination                                      
1      1                                     0.000000   
       2                                    13.516667   
       3                                    15.516667   
       4                                     7.366667   
       5                                     0.000000   
       6                                    11.366667   
       7                                     2.850000   
       8                                    10.000000   
       9                                     0.000000   
       10                                    0.000000   
       11                                   10.000000   
       12                                    0.000000   
       13                                    0.000000   
       14                                    9.833333   
2      1                                    14.816667   
       2                                     0.000000   
       3                                     4.750000   
       4                                     1.150000   
       5                                     0.000000   
       6                                     0.000000   
       7                                     0.000000   
       8                                     0.000000   
       9                                     0.000000   
       10                                    0.000000   
       11                                    0.000000   
       12                                    0.000000   
       13                                    4.616667   
       14                                    0.000000   
3      1                                    11.900000   
       2                                     4.783333   
       3                                     0.000000   
       4                                     6.683333   
       5                                     0.000000   
       6                                     2.250000   
       7                                     8.000000   
       8                                     5.000000   
       9                                     0.000000   
       10                                    0.000000   
       11                                    5.000000   
       12                                    9.283333   
       13                                    0.000000   
       14                                   18.500000   
4      1                                     6.500000   
       2                                     5.350000   
       3                                     9.400000   
       4                                     0.000000   
       5                                     0.000000   
       6                                     0.000000   
       7                                     0.000000   
       8                                     0.000000   
       9                                     0.000000   
       10                                    0.000000   
       11                                    0.000000   
       12                                    0.000000   
       13                                    0.000000   
       14                                    0.000000   

                    boardingdefault_noclamp_walkdefault  \
origin destination                                        
1      1                                       0.000000   
       2                                       7.666667   
       3                                       7.583333   
       4                                      -0.566667   
       5                                       0.000000   
       6                                       3.433333   
       7                                      -5.083333   
       8                                       2.066667   
       9                                       0.000000   
       10                                      0.000000   
     

#### comparison


In [3]:
original = pd.read_csv('/Users/admin/Desktop/bogota_urap/OTP/transMiCable/transMiCable_v0430/stop_times.txt')
original.drop(['pickup_type', 'drop_off_type', 'shape_dist_traveled', 'timepoint'])

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
0,133,04:53:43,04:53:53,0,1,NaN,NaN,NaN,NaN,NaN
1,133,04:59:53,05:00:04,1,2,NaN,NaN,NaN,NaN,NaN
2,133,05:04:04,05:04:15,2,3,NaN,NaN,NaN,NaN,NaN
3,133,05:08:15,05:08:25,3,4,NaN,NaN,NaN,NaN,NaN
4,133,05:12:25,05:12:36,2,5,NaN,NaN,NaN,NaN,NaN
